### **1.2.1 注意力机制实现的多尺度基准测试**
---

请编写一个脚本，在不同的尺度下对你实现的注意力机制进行基准测试。脚本应包含以下步骤：

*   **(a)** 将批大小（Batch Size）固定为 8，且不使用多头注意力机制（即：移除“头”维度，将其看作单头）。
*   **(b)** 遍历以下参数的笛卡尔积：头嵌入维度 $d_{model}$ 取值为 `[16, 32, 64, 128]`，序列长度（Sequence Length）取值为 `[256, 1024, 4096, 8192, 16384]`。
*   **(c)** 创建对应尺寸的随机输入张量 $Q, K, V$。
*   **(d)** 测量使用这些输入进行 100 次前向传播（Forward Passes）的时间。
*   **(e)** 在反向传播开始前，测量当前正在使用的显存量（Memory in use），并测量 100 次反向传播（Backward Passes）的时间。
*   **(f)** 确保进行预热（Warm up），并且在每次前向/反向传播之后调用 `torch.cuda.synchronize()`（以确保测速准确）。

**实验报告要求：**

汇报在这些配置下得到的耗时数据（如果报错则汇报显存溢出错误，即 Out-of-Memory, OOM）。
1.  在什么样的尺寸下会出现显存溢出（OOM）？
2.  针对你发现的出现 OOM 的最小配置之一，进行注意力机制显存占用的“账目计算”（你可以使用 Assignment 1 中关于 Transformer 显存占用的计算公式）。
3.  为了反向传播而保存（缓存）的显存量是如何随序列长度变化的？
4.  你会采取什么方法来消除这部分显存开销？

**交付物：**
一份包含耗时数据的表格、显存占用的计算推导过程，以及 1-2 段文字的回答。

---

In [ ]:
import os
import torch.nn.functional as F

def pytorch_naive_attention(q, k, v, mask=None):
    """
    实现朴素的 PyTorch Attention: softmax(Q @ K.T / sqrt(dk)) @ V
    
    参数:
        q: Query 张量，形状为 (batch_size, seq_len, d_model)
        k: Key 张量，形状为 (batch_size, seq_len, d_model)
        v: Value 张量，形状为 (batch_size, seq_len, d_model)
        mask: 可选的掩码张量 (例如 causal mask)
        
    返回:
        output: 注意力计算后的输出，形状为 (batch_size, seq_len, d_model)
    """
    #step1:calculate attentionScore
    attentionScore=q@k.transpose(-2,-1)

    #step2:normalize
    d_k=q.shape[-1]
    attentionScore=attentionScore/d_k**0.5

    #step3:mask
    if mask is not None:
        attentionScore=attentionScore.masked_fill(~mask,float('-inf'))

    #step4:softmax
    attentionScore=torch.softmax(attentionScore,dim=-1)

    #step5:calculate output
    output=attentionScore@v
    return output

In [ ]:
import time
import torch
import pandas as pd
import numpy as np

def benchmark_attention():
    batch_size = 8
    d_model_list = [16, 32, 64, 128]
    seq_len_list = [256, 1024, 4096, 8192, 16384]

    device = "cuda" if torch.cuda.is_available() else "cpu"
    results = []

    for d_model in d_model_list:
        for seq_len in seq_len_list:
            print(f"Testing: d_model={d_model}, seq_len={seq_len}...")
            
            # 清理显存缓存
            if device == "cuda":
                torch.cuda.empty_cache()
            
            try:
                # 构造数据，需要 requires_grad 来测试后向传播
                q = torch.randn(batch_size, seq_len, d_model, device=device, requires_grad=True)
                k = torch.randn(batch_size, seq_len, d_model, device=device, requires_grad=True)
                v = torch.randn(batch_size, seq_len, d_model, device=device, requires_grad=True)

                # --- 1. Warm up ---
                for _ in range(10):
                    _ = pytorch_naive_attention(q, k, v)

                # --- 2. Measure Forward Pass ---
                if device == "cuda": torch.cuda.synchronize()
                start_fwd = time.perf_counter()
                
                for _ in range(100):
                    out = pytorch_naive_attention(q, k, v)
                
                if device == "cuda": torch.cuda.synchronize()
                end_fwd = time.perf_counter()
                avg_fwd = (end_fwd - start_fwd) / 100 * 1000  # 转为 ms

                # --- 3. Measure Memory ---
                # 在后向传播开始前记录显存
                mem_usage = torch.cuda.memory_allocated(device) / (1024 ** 2) # MB

                # --- 4. Measure Backward Pass ---
                grad_output = torch.randn_like(out)
                if device == "cuda": torch.cuda.synchronize()
                start_bwd = time.perf_counter()
                
                for _ in range(100):
                    out.backward(grad_output, retain_graph=True)
                
                if device == "cuda": torch.cuda.synchronize()
                end_bwd = time.perf_counter()
                avg_bwd = (end_bwd - start_bwd) / 100 * 1000 # ms

                results.append({
                    "d_model": d_model,
                    "seq_len": seq_len,
                    "fwd_ms": f"{avg_fwd:.3f}",
                    "bwd_ms": f"{avg_bwd:.3f}",
                    "mem_mb": f"{mem_usage:.2f}"
                })

            except RuntimeError as e:
                if "out of memory" in str(e).lower():
                    print(f"OOM at seq_len={seq_len}")
                    results.append({
                        "d_model": d_model,
                        "seq_len": seq_len,
                        "fwd_ms": "OOM",
                        "bwd_ms": "OOM",
                        "mem_mb": "OOM"
                    })
                else:
                    raise e

    # 打印结果表格 (作业 1.1.2 建议使用 pandas 打印表格)
    df = pd.DataFrame(results)
    print("\nBenchmark Results:")
    print(df.to_markdown()) # 需要 pip install tabulate

if __name__ == "__main__":
    benchmark_attention()


Benchmark Results:
|    |   d_model |   seq_len | fwd_ms   | bwd_ms   | mem_mb   |
|---:|----------:|----------:|:---------|:---------|:---------|
|  0 |        16 |       256 | 0.118    | 6.040    | 10.62    |
|  1 |        16 |      1024 | 1.241    | 2.177    | 50.38    |
|  2 |        16 |      4096 | 15.336   | 29.282   | 536.75   |
|  3 |        16 |      8192 | 64.601   | 120.728  | 2082.25  |
|  4 |        16 |     16384 | OOM      | OOM      | OOM      |
|  5 |        32 |       256 | 0.102    | 0.431    | 23.25    |
|  6 |        32 |      1024 | 1.241    | 2.184    | 52.50    |
|  7 |        32 |      4096 | 16.176   | 30.388   | 545.25   |
|  8 |        32 |      8192 | 67.603   | 128.251  | 2100.25  |
|  9 |        32 |     16384 | OOM      | OOM      | OOM      |
| 10 |        64 |       256 | 0.108    | 0.307    | 28.25    |
| 11 |        64 |      1024 | 1.306    | 2.512    | 56.75    |
| 12 |        64 |      4096 | 19.095   | 34.864   | 562.25   |
| 13 |        64 |      8192 | 81.289   | 152.397  | 2136.25  |
| 14 |        64 |     16384 | OOM      | OOM      | OOM      |
| 15 |       128 |       256 | 0.201    | 0.423    | 38.25    |
| 16 |       128 |      1024 | 1.807    | 3.657    | 65.25    |
| 17 |       128 |      4096 | 27.715   | 52.034   | 596.25   |
| 18 |       128 |      8192 | 127.961  | 238.462  | 2208.25  |
| 19 |       128 |     16384 | OOM      | OOM      | OOM      |

# 实验报告要求：

汇报在这些配置下得到的耗时数据（如果报错则汇报显存溢出错误，即 Out-of-Memory, OOM）。

## Q1：在什么样的尺寸下会出现显存溢出（OOM）？
针对你发现的出现 OOM 的最小配置之一，进行注意力机制显存占用的“账目计算”（你可以使用 Assignment 1 中关于 Transformer 显存占用的计算公式）。
A:当seq_len=16384的时候，不管d_model大小，内存都会溢出

Q,K,V Matrix=batchSize*d_model*seq_len*3*4byte=8*16*16384*3*4=0.02GB

Forward:attentionScore=batchsize*seq_len*seq_len*4byte=8*16384*16384*4=8GB

Backward:保留前向的矩阵8GB

total=QKVMatrix+Forward+Backward>16GB（GPU T4*2）


# 1.3.0

---

### **问题 (torch_compile)：2分**

**(a)** 扩展你之前的注意力机制基准测试脚本，加入你所实现的 PyTorch 注意力机制的**编译版本（compiled version）**。在与上述 `pytorch_attention` 问题（即 1.2.1 节）相同的配置下，对比其与非编译版本的性能。

*   **交付物**：一张对比表格，比较编译后的注意力模块与之前 `pytorch_attention` 问题中非编译版本的**前向（forward）**和**反向（backward）**传播耗时。

**(b)** 现在，在你之前的端到端基准测试脚本（即 1.1.3 节中用于测试整个模型的脚本）中，**编译你的整个 Transformer 模型**。前向传播的性能发生了怎样的变化？前向传播、反向传播以及优化器步骤（optimizer steps）组合在一起的整体性能又发生了怎样的变化？

*   **交付物**：一张对比表格，比较**原始原生（vanilla）** Transformer 模型与**编译后（compiled）** Transformer 模型的性能。

---

### **💡 任务重点说明：**

#### **1. 针对 (a) 的实现：**
你不需要写新函数，只需要利用 `torch.compile`。
*   **做法**：在测试循环开始前，执行 `compiled_fn = torch.compile(pytorch_naive_attention)`。
*   **意义**：你会观察到，由于 `torch.compile` 尝试将矩阵乘法、除法、Mask、Softmax 这些操作“融合”进更少的 GPU 内核（Kernels）中，减少了显存读写次数，速度应该会有提升。

#### **2. 针对 (b) 的实现：**
这次是针对**整个模型类**。
*   **做法**：`compiled_model = torch.compile(model)`。
*   **测量内容**：这次不仅仅测 Attention 函数，还要测量一个完整的训练步（Training Step），包括：
    1.  `output = compiled_model(input)`（前向）
    2.  `loss.backward()`（反向）
    3.  `optimizer.step()`（优化器更新）
*   **意义**：全模型编译通常比单函数编译有更大的优化空间，因为它能跨层进行算子融合。

#### **3. ⚠️ 实验陷阱提醒：**
*   **预热（Warm-up）**：`torch.compile` 在**第一次执行**时会非常慢，因为它在后台调用 Triton 生成内核代码。你必须在计时循环开始前，先跑几次模型，确保编译完成。
*   **显存**：注意观察，虽然速度变快了，但在长序列（如 16384）下，编译版是否依然会报错 OOM？（答案通常是：是的，因为它依然要存 $N^2$ 的注意力矩阵）。


In [ ]:
import time
import torch
import pandas as pd
import numpy as np

def benchmark_attention():
    batch_size = 8
    d_model_list = [16, 32, 64, 128]
    seq_len_list = [256, 1024, 4096, 8192, 16384]
    
    if torch.cuda.is_available():
        device = "cuda"
        print(f"Is CUDA available: {torch.cuda.is_available()}")
        print(f"GPU Device Name: {torch.cuda.get_device_name(0)}")
        props = torch.cuda.get_device_properties(0)
        print(f"Total Memory: {props.total_memory / 1024**2:.2f} MB")
        print(f"Processor Count: {props.multi_processor_count}")
        print(f"Compute Capability: {props.major}.{props.minor}")
    else:
        device ="cpu"
    results = []

    compiled_naive_attention = torch.compile(pytorch_naive_attention)
    for d_model in d_model_list:
        for seq_len in seq_len_list:
            print(f"Testing: d_model={d_model}, seq_len={seq_len}...")
            
            # 清理显存缓存
            if device == "cuda":
                torch.cuda.empty_cache()
            
            try:
                # 构造数据，需要 requires_grad 来测试后向传播
                q = torch.randn(batch_size, seq_len, d_model, device=device, requires_grad=True)
                k = torch.randn(batch_size, seq_len, d_model, device=device, requires_grad=True)
                v = torch.randn(batch_size, seq_len, d_model, device=device, requires_grad=True)

                # --- 1. Warm up ---
                for _ in range(10):
                    warm_up_out = compiled_naive_attention(q, k, v)
                    warm_up_out.backward(torch.randn_like(warm_up_out))
                    
                # --- 2. Measure Forward Pass ---
                if device == "cuda": torch.cuda.synchronize()
                start_fwd = time.perf_counter()
                
                for _ in range(100):
                    out = compiled_naive_attention(q, k, v)
                
                if device == "cuda": torch.cuda.synchronize()
                end_fwd = time.perf_counter()
                avg_fwd = (end_fwd - start_fwd) / 100 * 1000  # 转为 ms

                # --- 3. Measure Memory ---
                # 在后向传播开始前记录显存
                mem_usage = torch.cuda.memory_allocated(device) / (1024 ** 2) # MB

                # --- 4. Measure Backward Pass ---
                grad_output = torch.randn_like(out)
                if device == "cuda": torch.cuda.synchronize()
                start = time.perf_counter()
                
                for _ in range(100):
                    out=compiled_naive_attention(q,k,v)
                    out.backward(grad_output)
                if device == "cuda": torch.cuda.synchronize()
                end = time.perf_counter()
                avg_fwd_bwd = (end - start) / 100 * 1000 # ms 注意这里包含前向和反向的时间
                real_bwd_ms=avg_fwd_bwd-avg_fwd

                results.append({
                    "d_model": d_model,
                    "seq_len": seq_len,
                    "fwd_ms": f"{avg_fwd:.3f}",
                    "bwd_ms": f"{real_bwd_ms:.3f}",
                    "mem_mb": f"{mem_usage:.2f}"
                })

            except RuntimeError as e:
                if "out of memory" in str(e).lower():
                    print(f"OOM at seq_len={seq_len}")
                    results.append({
                        "d_model": d_model,
                        "seq_len": seq_len,
                        "fwd_ms": "OOM",
                        "bwd_ms": "OOM",
                        "mem_mb": "OOM"
                    })
                else:
                    raise e

    # 打印结果表格 (作业 1.3.1 建议使用 pandas 打印表格)
    df = pd.DataFrame(results)
    print("\nBenchmark Results:")
    print(df.to_markdown()) # 需要 pip install tabulate

if __name__ == "__main__":
    benchmark_attention()

- Is CUDA available: True
- GPU Device Name: Tesla T4
- Total Memory: 14912.69 MB
- Processor Count: 40
- Compute Capability: 7.5

before complie:
|    |   d_model |   seq_len | fwd_ms   | bwd_ms   | mem_mb   |
|---:|----------:|----------:|:---------|:---------|:---------|
|  0 |        16 |       256 | 0.118    | 6.040    | 10.62    |
|  1 |        16 |      1024 | 1.241    | 2.177    | 50.38    |
|  2 |        16 |      4096 | 15.336   | 29.282   | 536.75   |
|  3 |        16 |      8192 | 64.601   | 120.728  | 2082.25  |
|  4 |        16 |     16384 | OOM      | OOM      | OOM      |
|  5 |        32 |       256 | 0.102    | 0.431    | 23.25    |
|  6 |        32 |      1024 | 1.241    | 2.184    | 52.50    |
|  7 |        32 |      4096 | 16.176   | 30.388   | 545.25   |
|  8 |        32 |      8192 | 67.603   | 128.251  | 2100.25  |
|  9 |        32 |     16384 | OOM      | OOM      | OOM      |
| 10 |        64 |       256 | 0.108    | 0.307    | 28.25    |
| 11 |        64 |      1024 | 1.306    | 2.512    | 56.75    |
| 12 |        64 |      4096 | 19.095   | 34.864   | 562.25   |
| 13 |        64 |      8192 | 81.289   | 152.397  | 2136.25  |
| 14 |        64 |     16384 | OOM      | OOM      | OOM      |
| 15 |       128 |       256 | 0.201    | 0.423    | 38.25    |
| 16 |       128 |      1024 | 1.807    | 3.657    | 65.25    |
| 17 |       128 |      4096 | 27.715   | 52.034   | 596.25   |
| 18 |       128 |      8192 | 127.961  | 238.462  | 2208.25  |
| 19 |       128 |     16384 | OOM      | OOM      | OOM      |


after compile:
|    |   d_model |   seq_len | fwd_ms   | bwd_ms   | mem_mb   |
|---:|----------:|----------:|:---------|:---------|:---------|
|  0 |        16 |       256 | 0.354    | 0.487    | 19.25    |
|  1 |        16 |      1024 | 1.315    | 1.227    | 52.38    |
|  2 |        16 |      4096 | 15.356   | 23.546   | 544.75   |
|  3 |        16 |      8192 | 65.147   | 98.962   | 2098.25  |
|  4 |        16 |     16384 | OOM      | OOM      | OOM      |
|  5 |        32 |       256 | 0.939    | 0.860    | 24.25    |
|  6 |        32 |      1024 | 3.713    | 4.169    | 56.50    |
|  7 |        32 |      4096 | 23.691   | 32.629   | 561.25   |
|  8 |        32 |      8192 | 80.472   | 124.365  | 2132.25  |
|  9 |        32 |     16384 | OOM      | OOM      | OOM      |
| 10 |        64 |       256 | 0.969    | 0.878    | 30.25    |
| 11 |        64 |      1024 | 3.990    | 4.636    | 64.75    |
| 12 |        64 |      4096 | 28.613   | 41.088   | 594.25   |
| 13 |        64 |      8192 | 102.531  | 168.196  | 2200.25  |
| 14 |        64 |     16384 | OOM      | OOM      | OOM      |
| 15 |       128 |       256 | 1.091    | 0.925    | 42.25    |
| 16 |       128 |      1024 | 4.598    | 5.939    | 81.25    |
| 17 |       128 |      4096 | 40.429   | 60.686   | 660.25   |
| 18 |       128 |      8192 | 147.319  | 250.266  | 2336.25  |
| 19 |       128 |     16384 | OOM      | OOM      | OOM      |

### 1.3.1 Example - Weighted Sum (Triton 入门)
---
**它是做什么的：**
如何用 Triton 编写一个简单的内核：计算矩阵 $X$ 每一行与向量 $w$ 的点积。

**为什么要先看这个：**
*   **理解“分块指针” (Block Pointers)**：这是 Triton 的核心。以前写 CUDA 算索引很痛苦（要算 `threadIdx`, `blockIdx` 等），Triton 使用 `tl.make_block_ptr` 让你像操作切片一样操作 GPU 内存。
*   **理解“步进” (Advance)**：学习如何用 `ptr.advance` 在内存中移动你的窗口。
*   **代码封装**：学习如何把 Triton 内核包装进 `torch.autograd.Function` 的 `forward` 和 `backward` 中，使其能无缝融入 PyTorch 的训练流程。

**你应该怎么做：**
1.  **不要跳过阅读**：仔细阅读 PDF 第 10-14 页的代码。
2.  **在 Kaggle 手敲一遍**：尝试运行 PDF 中提供的 `weighted_sum` 代码。
3.  **理解反向传播**：看懂梯度 $dw$ 和 $dx$ 是如何在 Triton 循环里累加的。

---

In [ ]:
#1.3.1 Example - Weighted Sum
def weighted_sum(x,weight):
    # 此处假设 x 的维度是 [..., D]，weight 的维度是 [D],维度不匹配， 广播+点积
    return (weight *x).sum(axis=-1)





import triton
import triton.language as tl

import torch
#前向传播
@triton.jit
def weight_sum_fwd(
    x_ptr,weight_ptr, #输入指针
    output_ptr, #输出指针
    x_row_stride,x_stride_dim,#步长，告诉我们如何在张量的每个轴上移动一个元素
    weight_stride_dim,#通常为1
    output_stride_row,#通常为1
    ROWS,D,#矩阵总行数和维度
    ROWS_TILE_SIZE:tl.constexpr,
    D_TILE_SIZE: tl.constexpr,#分块大小：必须在编译时已知 对于3*2的矩阵ROWS_TILE_SIZE=1，D_TILE_SIZE=2
):
    #每个实例将计算一组行分块的加权和
    # tl.program_id(0) 指示当前在哪个线程块
    row_tile_idx=tl.program_id(0)
    
    #块指针（Block pointers） 让我们能选择内存中的一个N维区域并移动它
    """
    块指针需要知道：
    - 指向张量的第一个元素的指针
    - 张量的总形状（以处理越界访问）
    - 每个维度的步长（以正确使用内存布局）
    - 起始块的N维坐标(offsets)
    - 每次加载/存储时使用的块形状
    - 维度在内存中的排列顺序
    """
    x_block_ptr=tl.make_block_ptr(
        x_ptr,
        shape=(ROWS,D,),
        strides=(x_row_stride,x_stride_dim),
        offsets=(row_tile_idx * ROWS_TILE_SIZE,0),
        block_shape=(ROWS_TILE_SIZE,D_TILE_SIZE),
        order=(1,0),
    )

    weight_block_ptr=tl.make_block_ptr(
        weight_ptr,
        shape=(D,),
        strides=(weight_stride_dim,),
        offsets=(0,),
        block_shape=(D_TILE_SIZE,),
        order=(0,),
    )

    output_block_ptr=tl.make_block_ptr(
        output_ptr,
        shape=(ROWS,),
        strides=(output_stride_row,),
        offsets=(row_tile_idx * ROWS_TILE_SIZE,),
        block_shape=(ROWS_TILE_SIZE,),
        order=(0,),
    )
    #初始化一个缓冲区用于写入结果
    output=tl.zeros((ROWS_TILE_SIZE,),dtype=tl.float32)
    #在D维度上进行循环
    for i in range(tl.cdiv(D,D_TILE_SIZE)):
        #加载当前的块指针内容并带有边界检查
        row=tl.load(x_block_ptr,boundary_check=(0,1),padding_option="zero")
        weight=tl.load(weight_block_ptr,boundary_check=(0,),padding_option="zero")

        #计算该行分块的加权和
        output+=tl.sum(row*weight[None,:],axis=1)

        #将指针移动到下一个分块
        x_block_ptr=x_block_ptr.advance((0,D_TILE_SIZE))
        weight_block_ptr=weight_block_ptr.advance((D_TILE_SIZE,))
    #将结果写回输出快指针
    tl.store(output_block_ptr,output,boundary_check=(0,))

In [2]:
#反向传播
@triton.jit
def weighted_sum_backward(
    x_ptr,weight_ptr, #input
    grad_output_ptr, # Grad input
    grad_x_ptr,partial_grad_weight_ptr,#Grad outputs
    stride_xr,stride_xd,
    stride_wd,
    stride_gr,
    stride_gxr,stride_gxd,
    stride_gwb,stride_gwd,
    NUM_ROWS,D,#3,2
    ROWS_TILE_SIZE:tl.constexpr,D_TILE_SIZE:tl.constexpr,#Tile Size 必须是 2 的幂，ROWS_TILE_SIZE=1，D_TILE_SIZE=2
):
    row_tile_idx=tl.program_id(0)
    n_row_tiles=tl.num_programs(0)

    #计算X的梯度wj⋅gi
    #计算weight的梯度sum(xij*gi)
    #Inputs X=3*2
    #weight=2*1
    #weight_sum/ouput/gradout=3*1
    """
    场景:假设3*2矩阵计算完后.得到了一个长度为 3 的输出向量y
    输入梯度：现在反向传播传回了一个长度为 3 的梯度向量 grad_output = [g0, g1, g2]
    """
    grad_output_block_ptr=tl.make_block_ptr(
        base=grad_output_ptr,#地图的起点：整个向量在显存里的“门牌号”起点
        shape=(NUM_ROWS,),#地图的全长：告诉 GPU 这条数据总共有多长，别跑丢了
        strides=(stride_gr,),#步长：从一个梯度值走到下一个梯度值，需要跨过多少个地址空间
        offsets=(row_tile_idx*ROWS_TILE_SIZE,),#当前工人的起始点：工人i应该从第几个梯度开始看
        block_shape=(ROWS_TILE_SIZE,),#手电筒的宽度：工人一次性能看几个梯度值？
        order=(0,),#扫描方向：数据是按什么顺序排的？（1D 只有一种排法）
    )
    x_block_ptr=tl.make_block_ptr(
        base=x_ptr,
        shape=(NUM_ROWS,D,),
        strides=(stride_xr,stride_xd),
        offsets=(row_tile_idx*ROWS_TILE_SIZE,0),
        block_shape=(ROWS_TILE_SIZE,D_TILE_SIZE),
        order=(1,0),
    )
    weight_block_ptr=tl.make_block_ptr(
        base=weight_ptr,
        shape=(D,),#如果你的 shape 只有一个数（一维），那么你的 offsets 和 block_shape 也只能有一个数
        strides=(stride_wd,),
        offsets=(0,),
        block_shape=(D_TILE_SIZE,),
        order=(0,),
    )
    grad_x_block_ptr=tl.make_block_ptr(
        base=grad_x_ptr,
        shape=(NUM_ROWS,D,),
        strides=(stride_gxr,stride_gxd),
        offsets=(row_tile_idx*ROWS_TILE_SIZE,0),
        block_shape=(ROWS_TILE_SIZE,D_TILE_SIZE),
        order=(1,0),
    )
    partial_grad_weight_block_ptr=tl.make_block_ptr( #根据公式 (3)，权重 w的梯度是所有行累加的结果。由于每个工人处理不同的行，他们每个人都会算出一个对w的“局部贡献”,所以让每个工人把自己的计算结果写到一个叫 partial_grad_weight_ptr 的缓冲区里
        base=partial_grad_weight_ptr,
        shape=(NUM_ROWS,D,),
        strides=(stride_gwb,stride_gwd),
        offsets=(row_tile_idx,0),
        block_shape=(1,D_TILE_SIZE),
        order=(1,0),

    )
    for i in range(tl.cdiv(D,D_TILE_SIZE)):
        # 1. 加载输出梯度：工人 1 加载 g1。形状为 (1,)
        grad_output=tl.load(grad_output_block_ptr,boundary_check=(0,),padding_option="zero") #(ROWS_TILE_SIZE,)
        # 2. 加载权重：加载 [w0, w1]。形状为 (2,)
        weight=tl.load(weight_block_ptr,boundary_check=(0,),padding_option="zero")#(D_TILE_SIZE,)
        
        #3.计算X的梯度wj⋅gi 在 Triton和 PyTorch中，None 的作用是增加一个维度，以便触发广播机制
        grad_x_row=grad_output[:,None]*weight[None,:]
        # 4. 存储 X 的梯度：把 [[g1*w0, g1*w1]] 写回到 grad_x 矩阵的第 2 行
        tl.store(grad_x_block_ptr,grad_x_row,boundary_check=(0,1))

        # 5. 计算 weight 的部分梯度 (公式 3): (nabla_w)_j = sum(x_ij * g_i)
        row=tl.load(x_block_ptr,boundary_check=(0,1),padding_option="zero")#(ROWS_TILE_SIZE,D_TILE_SIZE)
        grad_weight_row=tl.sum(row*grad_output[:,None],axis=0,keep_dims=True)
        tl.store(partial_grad_weight_block_ptr,grad_weight_row,boundary_check=(1,))# Never out of bounds for dim 0
        
        # Move the pointers to the next tile along D
        x_block_ptr=x_block_ptr.advance((0,D_TILE_SIZE))
        weight_block_ptr=weight_block_ptr.advance((D_TILE_SIZE,))
        partial_grad_weight_block_ptr=partial_grad_weight_block_ptr.advance((0,D_TILE_SIZE))
        grad_x_block_ptr=grad_x_block_ptr.advance((0,D_TILE_SIZE))

为了彻底看清这行代码在 GPU 寄存器里是如何运算的，我们带入具体数值。

### 1. 场景设定
*   **输入权重 $weight$**: `[7, 8]`
*   **假设输出梯度 $grad\_output$**: `[1, 2, 3]` （我们假设这三个数是反向传播传回来的）
*   **当前工人**: 工人 1（`row_tile_idx = 1`），他负责处理 $X$ 的**第 2 行**（即数值为 `[3, 4]` 的那一行）。

---

### 2. 逐步计算过程

#### 第一步：加载数据（Load）
工人 1 从显存中通过 `tl.load` 拿到属于自己的那部分数据：
*   **`grad_output`**: 拿到第 1 个元素（索引从0开始），数值为 **`2`**。
*   **`weight`**: 拿到完整的权重，数值为 **`[7, 8]`**。

#### 第二步：维度重塑（Unsqueeze/None）
在 Triton（和 PyTorch）中，`None` 的作用是增加一个维度，以便触发**广播机制**。

1.  **`grad_output[:, None]`**:
    *   原形状：`[1]`（标量/1D向量）
    *   新形状：`(1, 1)`（列向量）
    *   **数值：`[[2]]`**

2.  **`weight[None, :]`**:
    *   原形状：`[2]`（1D向量）
    *   新形状：`(1, 2)`（行向量）
    *   **数值：`[[7, 8]]`**

#### 第三步：逐元素相乘（Outer Product / 外积）
代码：`grad_x_row = grad_output[:, None] * weight[None, :]`

这就是一个 **(1, 1)** 的矩阵 乘以一个 **(1, 2)** 的矩阵：
$$ \begin{bmatrix} 2 \end{bmatrix} \times \begin{bmatrix} 7 & 8 \end{bmatrix} $$

根据广播规则，`[[2]]` 会被自动复制成 `[[2, 2]]` 来匹配宽度，然后进行对应相乘：
*   第一个位置：$2 \times 7 = 14$
*   第二个位置：$2 \times 8 = 16$

**计算结果 `grad_x_row`**: **`[[14, 16]]`**

---

### 3. 结果的物理含义

这个结果 `[[14, 16]]` 就是损失函数对 $X$ 矩阵中**第 2 行**两个元素的梯度：

*   对于元素 `3` ($x_{10}$)：它的梯度是 **14**。
*   对于元素 `4` ($x_{11}$)：它的梯度是 **16**。

**工人 1 最后的动作**：
他调用 `tl.store(grad_x_block_ptr, grad_x_row)`，把这组数 `[14, 16]` 存回到显存中 `grad_x` 矩阵的第二行位置。

---

### 4. 为什么这叫“外积”？

如果你把三个工人的计算结果合在一起看，你会发现整个 `grad_x` 矩阵其实就是两个向量的乘积：

$$ \nabla_x L = \begin{bmatrix} g_0 \\ g_1 \\ g_2 \end{bmatrix} \times \begin{bmatrix} w_0 & w_1 \end{bmatrix} = \begin{bmatrix} 1 \cdot 7 & 1 \cdot 8 \\ 2 \cdot 7 & 2 \cdot 8 \\ 3 \cdot 7 & 3 \cdot 8 \end{bmatrix} = \begin{bmatrix} 7 & 8 \\ 14 & 16 \\ 21 & 24 \end{bmatrix} $$

*   **前向传播**：是每一行和 $w$ 做**内积**（点积），变成一个数。
*   **反向传播**：是输出梯度和 $w$ 做**外积**，重新变回一个矩阵。




继续沿用 $3 \times 2$ 案例，看看**工人 1**（负责第 2 行）是如何计算权重 $w$ 的梯度的。

### 1. 场景设定
*   **输入矩阵 $X$ 的第 2 行 (`row`)**: `[3, 4]`
*   **输出梯度 $grad\_output$ ($g$) 的第 2 个值**: `2`
*   **权重 $w$ 的长度**: 2（对应 $w_0, w_1$）

---

### 2. 逐步计算过程

#### 第一步：加载数据（Load）
工人 1 通过指针拿到他需要的“原材料”：
*   **`row`**: 拿到 $X$ 的第 1 行（索引从0开始），数值为 **`[3, 4]`**。
*   **`grad_output`**: 拿到输出梯度的第 1 个值，数值为 **`2`**。

#### 第二步：准备乘法维度（Broadcasting）
代码：`row * grad_output[:, None]`

1.  **`row`**: 形状是 `(1, 2)`，数值是 `[[3, 4]]`。
2.  **`grad_output[:, None]`**: 形状是 `(1, 1)`，数值是 `[[2]]`。

进行逐元素相乘时，`[[2]]` 会广播（复制）成 `[[2, 2]]`：
*   第一个位置：$3 \times 2 = 6$
*   第二个位置：$4 \times 2 = 8$

**得到中间结果**：**`[[6, 8]]`**。
这个结果的物理含义是：**“仅根据第 2 行的数据，我认为 $w_0$ 应该更新 6， $w_1$ 应该更新 8”**。

#### 第三步：局部求和（Reduction）
代码：`grad_weight_row = tl.sum(..., axis=0, keep_dims=True)`

*   因为我们的 `ROWS_TILE_SIZE = 1`（每个工人只管一行），所以这一步在 `axis=0`（行方向）上求和，其实就是它自己。
*   **结果 `grad_weight_row`**: **`[[6, 8]]`**。

> **注：** 如果 `ROWS_TILE_SIZE = 16`，这个工人会一次性读入 16 行，那么这步 `tl.sum` 就会把这 16 行对 $w$ 的贡献全部加起来，缩减成一行 `[dw0, dw1]`。

#### 第四步：存储到局部缓冲区（Partial Store）
代码：`tl.store(partial_grad_weight_block_ptr, grad_weight_row)`

工人 1 将他的结果 `[6, 8]` 写到 `partial_grad_weight` 这个大矩阵的**第 1 行**（对应他的工号）。

---

### 3. 全局视角：三个工人干完活后的样子

当所有工人（0, 1, 2）都完成上述步骤后，`partial_grad_weight` 矩阵在显存里长这样：

| 工人编号 | 计算过程 | 局部梯度结果 (写在矩阵对应的行) |
| :--- | :--- | :--- |
| **工人 0** | $g_0 \times [x_{00}, x_{01}] = 1 \times [1, 2]$ | **`[1, 2]`** |
| **工人 1** | $g_1 \times [x_{10}, x_{11}] = 2 \times [3, 4]$ | **`[6, 8]`** |
| **工人 2** | $g_2 \times [x_{20}, x_{21}] = 3 \times [5, 6]$ | **`[15, 18]`** |

---

### 4. 最终汇总（在内核之外）

现在，显存里有一个 $3 \times 2$ 的 `partial_grad_weight` 矩阵。但 PyTorch 最终需要的 $\nabla_w L$ 只是一个长度为 2 的向量。

于是，在 Python 包装类的 `backward` 函数中，你会看到类似这样的一行：
```python
# 把所有工人的局部贡献加起来，得到最终的权重梯度
grad_weight = partial_grad_weight.sum(axis=0)
```
**计算最终结果：**
*   $w_0$ 的总梯度 $= 1 + 6 + 15 = \mathbf{22}$
*   $w_1$ 的总梯度 $= 2 + 8 + 18 = \mathbf{28}$

---

### 💡 为什么不直接在 Triton 里加到一个向量上？

你可能会问：为什么工人们不直接往同一个 `grad_weight` 向量里加？

1.  **竞争（Race Condition）**：如果工人 0 和工人 1 同时计算完，都要写同一个内存地址，会发生冲突，导致结果错误。
2.  **原子操作太慢**：虽然可以使用 `tl.atomic_add` 来强行写入，但原子操作会让 GPU 线程排队，导致性能大幅下降。
3.  **分治法（Map-Reduce）**：让每个工人写自己的位置（Map），最后统一求和（Reduce），这是高性能计算中最标准、最高效的套路。

In [5]:
import torch
import triton
import triton.language as tl
from einops import rearrange

def cdiv(a, b):
    return (a + b - 1) // b

class WeightedSumFunc(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x, weight):
        # 1. 预处理
        ctx.original_x_shape = x.shape  # 保存矩阵原始形状
        input_shape = x.shape
        D = input_shape[-1]
        x_flattened = rearrange(x, "... d -> (...) d").contiguous()
        weight = weight.contiguous()
        
        n_rows = x_flattened.shape[0]
        y = torch.empty((n_rows,), device=x.device, dtype=x.dtype)
        
        # 2. 块大小设置
        # 注意：D_TILE_SIZE 必须是 2 的幂且不能超过 D
        D_TILE_SIZE = min(128, triton.next_power_of_2(D)) 
        ROWS_TILE_SIZE = 16
        
        # 3. 调用内核
        grid = (cdiv(n_rows, ROWS_TILE_SIZE), )
        weight_sum_fwd[grid](
            x_ptr=x_flattened, 
            weight_ptr=weight, 
            output_ptr=y,
            x_row_stride=x_flattened.stride(0), 
            x_stride_dim=x_flattened.stride(1),
            weight_stride_dim=weight.stride(0),
            output_stride_row=1, # 【修正点 1】一维向量输出步长固定为 1
            ROWS=n_rows, D=D,
            ROWS_TILE_SIZE=ROWS_TILE_SIZE,
            D_TILE_SIZE=D_TILE_SIZE,
        )
        
        ctx.save_for_backward(x_flattened, weight)
        ctx.ROWS_TILE_SIZE = ROWS_TILE_SIZE
        ctx.D_TILE_SIZE = D_TILE_SIZE
        
        return y.view(input_shape[:-1])

    @staticmethod
    def backward(ctx, grad_out):
        x, weight = ctx.saved_tensors
        n_rows, D = x.shape
        ROWS_TILE_SIZE = ctx.ROWS_TILE_SIZE
        D_TILE_SIZE = ctx.D_TILE_SIZE
        
        # 展平 grad_out 以匹配 1D 逻辑
        grad_out = grad_out.reshape(-1).contiguous()
        
        grad_x = torch.empty_like(x)
        n_row_tiles = cdiv(n_rows, ROWS_TILE_SIZE)
        partial_grad_weight = torch.zeros((n_row_tiles, D), device=x.device, dtype=x.dtype)

        grid = (n_row_tiles, )
        weighted_sum_backward[grid](
            x_ptr=x, weight_ptr=weight,
            grad_output_ptr=grad_out,
            grad_x_ptr=grad_x, 
            partial_grad_weight_ptr=partial_grad_weight,
            stride_xr=x.stride(0), stride_xd=x.stride(1),
            stride_wd=weight.stride(0),
            stride_gr=grad_out.stride(0),
            stride_gxr=grad_x.stride(0), stride_gxd=grad_x.stride(1),
            stride_gwb=partial_grad_weight.stride(0), stride_gwd=partial_grad_weight.stride(1),
            NUM_ROWS=n_rows, D=D,
            ROWS_TILE_SIZE=ROWS_TILE_SIZE, D_TILE_SIZE=D_TILE_SIZE,
        )
        
        grad_weight = partial_grad_weight.sum(axis=0)
        return grad_x.view(ctx.original_x_shape), grad_weight

In [8]:
import os; 
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
# 1. 实例化
f_weightedsum = WeightedSumFunc.apply

# 2. 准备数据 (requires_grad=True 才能测反向)
batch, seq, d = 8, 512, 128
x = torch.randn(batch, seq, d, device='cuda', requires_grad=True)
w = torch.randn(d, device='cuda', requires_grad=True)

# 3. 运行Triton 算子 (Forward)
output_triton = f_weightedsum(x, w)

# --- 验证点 1: 检查 grad_fn ---
print(f"Output Tensor: {output_triton}")
print(f"Attached grad_fn: {output_triton.grad_fn}") 
# 这里你应该能看到 <WeightedSumFuncBackward>，这证明 PyTorch 已经“记住”了你的算子

# --- 验证点 2: 检查结果正确性 (与原生 PyTorch 对比) ---
output_pytorch = (w * x).sum(dim=-1)
# 允许微小的浮点误差
is_correct = torch.allclose(output_triton, output_pytorch, atol=1e-5)
print(f"Forward Correctness: {is_correct}")

# --- 验证点 3: 检查反向传播 (Backward) ---
# 假设对 output 的总和求导
loss = output_triton.sum()
loss.backward()

print(f"X gradient calculated: {x.grad is not None}")
print(f"Weight gradient calculated: {w.grad is not None}")

# 对比 X 的梯度
x_grad_triton = x.grad.clone()
x.grad.zero_()
(w * x).sum().backward()
print(f"Backward Correctness: {torch.allclose(x_grad_triton, x.grad, atol=1e-5)}")

Output Tensor: tensor([[  8.5580,  -5.5923,  10.1044,  ...,  -2.9318,  16.2322,  -9.8391],
        [ 31.9812,   6.1009,   2.6397,  ..., -12.8501,  10.3944,  -3.1172],
        [-11.6288,  18.4553, -32.0443,  ...,   6.1585,   0.8947,   5.9122],
        ...,
        [  4.3335,  -6.8354,  -3.6885,  ..., -23.5979, -18.3236, -15.0220],
        [ 12.0635,   4.7010, -21.5463,  ...,  -2.5938,  -5.2389,   8.9815],
        [-18.2658,   9.1820,   5.1162,  ...,   9.7913, -16.5786,   0.5825]],
       device='cuda:0', grad_fn=<WeightedSumFuncBackward>)
Attached grad_fn: <torch.autograd.function.WeightedSumFuncBackward object at 0x79e3e8977790>
Forward Correctness: True
X gradient calculated: True
Weight gradient calculated: True
Backward Correctness: True
